# Machine Learning: Classification 

(Still some issues with this notebook)


In [ ]:
%pylab inline
from ggplot import *

# Enable inline plotting


from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [ ]:
data = load_iris()
# The data has some meta data appended at the top of the file
# The data is stored in the array "data"
# The field names are stored in "feature_names"
# Acutal flower names are stored in "target"
col_names = data['feature_names']

df = pd.DataFrame(data['data'], columns = col_names)
# Add in the Iris Id data
df['Iris Flower'] = data['target']
df.info()

Let's quickly re-name the **Iris flower** from an integer to the actual names as strings.

In [ ]:
df.head()

In [ ]:
df['Iris Flower'][df['Iris Flower']== 0] = 'Setosa' 
df['Iris Flower'][df['Iris Flower']== 1] = 'Versicolor' 
df['Iris Flower'][df['Iris Flower']== 2] = 'Virginica' 
# Print unique names to test if it worked properly
print pd.unique(df["Iris Flower"])

Let's get a quick look at our data so far.

In [ ]:
df.head()

In [ ]:
df.describe()

It's always a good idea to start plotting early to get a deeper sense of the data.

In [ ]:
from pandas.tools.plotting import scatter_matrix

scatter_matrix(df, figsize=[12,8])

This looks alright, but really we want to see the different flowers as separate colors, which is not easily possible right now. I did however find a keen <a href = ""http://stackoverflow.com/users/1586229/bgschiller>"Stack Overflow user</a> who developed a custom version for now.

In [ ]:
def factor_scatter_matrix(df, factor, palette=None):

    import matplotlib.colors
    from scipy.stats import gaussian_kde

    if isinstance(factor, basestring):
        factor_name = factor #save off the name
        factor = df[factor] #extract column
        df = df.drop(factor_name,axis=1) # remove from df, so it 
        # doesn't get a row and col in the plot.

    classes = list(set(factor))

    if palette is None:
        palette = ['#e41a1c', '#377eb8', '#4eae4b', 
                   '#994fa1', '#ff8101', '#fdfc33', 
                   '#a8572c', '#f482be', '#999999']

    color_map = dict(zip(classes,palette))

    if len(classes) > len(palette):
        raise ValueError('''Too many groups for the number of colors provided.
We only have {} colors in the palette, but you have {}
groups.'''.format(len(palette), len(classes)))

    colors = factor.apply(lambda group: color_map[group])
    axarr = scatter_matrix(df,figsize=(10,10),marker='o',c=colors,diagonal=None)


    for rc in xrange(len(df.columns)):
        for group in classes:
            y = df[factor == group].icol(rc).values
            gkde = gaussian_kde(y)
            ind = np.linspace(y.min(), y.max(), 1000)
            axarr[rc][rc].plot(ind, gkde.evaluate(ind),c=color_map[group])

    return axarr, color_map

In [ ]:
axarr, color_map = factor_scatter_matrix(df,'Iris Flower')

**ITS BEAUTIFUL**

On the subject of useful plotting methods, <a href = "http://stanford.edu/~mwaskom/software/seaborn/tutorial/axis_grids.html#plotting-bivariate-data-with-jointgrid"> Seaborn</a> is producing some really cool stuff. I find the scatterplot + histograms really useful.

Anyways, ggplot can at least take care of the rest of out plotting needs.

In [ ]:
print(ggplot(df, aes(x="petal length (cm)",y="petal width (cm)", color = "Iris Flower"))+geom_point())
print(ggplot(df, aes(x="sepal length (cm)",y="sepal width (cm)", color = "Iris Flower"))+geom_point())

# The simplest way to classify

Our goal is to develop a model to classify the three different types of flowers. 

By inspecting the graphs above, we can see that there is actually a very clear separation in petal length and width between Iris Setosa and the other two flowers. 

We could write some quick code to classify that.

In [ ]:
# Determine the cutoffs for Iris Setosa petal length
setosa = df['Iris Flower'] == "Setosa"

max_setosa = df['petal length (cm)'][setosa].max()
print "Setosa max petal length: {} cm".format(max_setosa)

# Find the minimum petal length for the other plants
min_not_setosa = df['petal length (cm)'][-setosa].min()
print "Not Setosa min petal length: {} cm".format(min_not_setosa)

Once we have this classification scheme, we can create a very simply model to identify the type of flower.

In [ ]:
petal_length = df['petal length (cm)']
#Incoming wall of text
for flower in petal_length:
    if flower <= 1.9:
        print("Iris Setosa")
    else:
        print("Iris Virginica or Iris Versicolour")

<hr>
That was less than exciting. This was just a simple example of a **manually implemented dimension threshold**. Eventually, we will get some **machine learning** to do this for us.

We can see that we will never attain perfect separation between Iris Virginica and Iris Versicolour, since there is some degree of overlap between them.

However, we can still try to implement another threshold.

In [ ]:
# First we select non-Setosa flowers
flowers = df[-setosa]

# Create a boolean mask of Iris Virginica to test prediction against
is_virginica = flowers['Iris Flower'] == 'Virginica'

# Convert to plain matrix for this simple looping operation
flowers = flowers.as_matrix()

Now we are going to loop through each of the petal and sepal features in the data and test each value as a potential threshold. At the same time we will keep track of the threshold with the best prediction accuracy.

In [ ]:
best_accuracy = [0, 0, 0]  # accuracy, feature, threshold
# for each flower feature in the data (columns 1 - 4)
# sepal length (cm) = 0 	
# sepal width (cm) = 1	
# petal length (cm)	= 2
# petal width (cm) = 3
for feature in xrange(4):
     
    threshold = flowers[:,feature].copy()
    threshold.sort()
    
    # Now test all these values as thresholds
    for thresh in threshold:
        prediction = flowers[:,feature] > thresh
        accuracy = (prediction == is_virginica).mean()
        if accuracy > best_accuracy[0]:
            best_accuracy = [accuracy, feature, thresh]
print best_accuracy

Great, we can see that we found our best flower classifying accuracy was 94% using petal width at 1.6 cm.

Lets see how this looks on a plot.

In [ ]:
# Select non-Setosa flowers
flowers = df[-setosa]
ggplot(flowers, aes(x="petal width (cm)", y="petal length (cm)", \
                    color = "Iris Flower")) + \
geom_point() + \
geom_vline(xintercept = 1.6, color = "black")

Hey! Not bad. **BUT**

# Testing / Training data and Cross-Validation

Our previous model achieved 94% accuracy, however, this is likely over-estimated. We used the same data to both define the threshold, and then evaluate the model.

What we should be doing is generalize the power of our model on new instances of data. In other words, we will define the threshold on a **training set** of data, and evaluate the model on the **test set** of data that the model did not see in training.

It is therefore only appropriate to report training error, not testing error, when communicating your results.

For now, we will split the data in half to achieve this, though more consideration should be taken later, in more complex models. Sometimes the models can be sensitive to how much data we include in either set. For example: sometimes too little data for testing could skew the result.

To avoid this problem, it would be nice to somehow use all the data for both testing and training.

This is where **cross-validation** comes in. There are many types of cross validation, but the most extreme case is **leave-one-out**. Under this framework, if we have 100 points of data in our training set, we:

- remove *i* th element of the training set
- train model using the training set
- predict model with testing set, where testing = training - *i* th element
- record the error
- repeat until all *i* elements of training set have been omitted (one at a time)
- sum error

Generally, we don't use leave-one-out unless we need particularly high precision, as other techniques are sufficient for most cases. The biggest caveat of "leave-one-out" cross-validation is dramatic increases in computation speed, particularly as dataset become large.


You can read up on the other types of cross validation <a href = "http://en.wikipedia.org/wiki/Cross-validation_(statistics)">here</a>.

## Getting a feel for testing and training data

Using pandas, we can efficiently split the data into testing and training sets.

Now let's modify our model definition from before and create a more formal function.

In [ ]:
def model(input_data):
    
    # Determine the row index's for the testing set
    test_rows = np.random.choice(input_data.index, size=50, replace = False)
    # Use this as a mask on the full dataset

    # Use .ix to specify array mask
    testing_data = input_data.ix[test_rows]
    training_data = input_data.drop(test_rows)
    
    # Create a boolean mask of Iris Virginica to test prediction against
    is_virginica = testing_data['Iris Flower'] == 'Virginica'

    # Convert to plain matrix for this simple looping operation
    training_data = training_data.as_matrix()
    
    best_accuracy = [0, 0, 0]  # accuracy, feature, threshold
    # for each flower feature in the data (columns 1 - 4)
    # sepal length (cm) = 0 	
    # sepal width (cm) = 1	
    # petal length (cm)	= 2
    # petal width (cm) = 3
    for feature in xrange(4):

        threshold = training_data[:,feature].copy()
        threshold.sort()

        # Now test all these values as thresholds
        for thresh in threshold:
            prediction = training_data[:,feature] > thresh
            accuracy = (prediction == is_virginica).mean()
            
            if accuracy > best_accuracy[0]:
                best_accuracy = [accuracy, feature, thresh]
    
    return best_accuracy

print "Best model accuracy = {}".format(model(flowers)[0])

As you can see, our results are much worse than before, but that's ok. We will be moving on to more sophisticated techniques soon.

Before that, let's check how sensitive our model is to the random sampling of our training and testing data.

In [ ]:
results_l = []
i = 0

while i < 1000:
    results = model(flowers)[0]
    results_l.append(results)
    i+=1

qplot(results_l)

After 1000 runs we can start to get a pretty good picture of the potential variation. Generally, we will let our machine learning package handle this type of thing for us though.

# K-Nearest Neighbor Classifier

Let's get into more complex classifiers. We will also be using a new agriculturally based dataset called <a href='http://archive.ics.uci.edu/ml/datasets/seeds#'>Seeds</a>, which consists of measurements of wheat seeds.

Pandas can load the data right from a url. Note that we had to exclude come rows due to data errors.

In [ ]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt"

df = pd.read_csv(url, sep = "\t", header=False, error_bad_lines=False,
names = ['area','perim','compact','length','width','asymc','length_groove', 'wheat_type'])

In [ ]:
df.describe()

As we can see, this dataset is a bit more complex than `iris`. We now have 7 features and three types of wheat: `Canadian`, `Koma`, and `Rosa`.

Of these 7 features, one of them, **compactness**, is not technically a new variable, but a function of two features, `perimeter` and `area`. It is common practise, and often very useful to derive new combined features like this. It usually termed **feature engineering**. 

The target for a good feature in any type of machine learning is that it is both varies with what matters and is invariate with what does not. Compactness in wheat varies with shape, and not size, which is a better predictor of wheat type. The best tool for creating good features is strong intuition and knowledge of the data. Generally, for most common sources of data, there are already a set of preferred and potential features generated by previous literature.

**All in all, simpler is better.**

## Definition:

K-nearest neighbour (KNN) is a supervised learning algorithm for classification or regression. One of the most common usages is for personalization tasks. To make an accurate personalized offer to a customer, you could use KNN to find similar customers and base your offer from their previous purchase behaviour.

<hr>
KNN is utilized when class labels are known for a dataset and you want to predict the class of a new data point. The predicted class will be based on the known classes NN. We are simply trying to find the class that is most similar to the new data point you are trying to predict.

For example: what class should the green circle belong to? Blue or Red?

In [ ]:
from IPython.core.display import Image
Image("http://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/200px-KnnClassification.svg.png")

## Choosing the right value of k

It turns out that  the correct value for `k` is very important. Generally, it is best to start out with an odd number, since this avoids situations where your classifier "ties" as a result of having the same number of votes for two or more classes. Like before in the IPython notebook for <a href = "">Curve Fitting (add link)</a>, we are vulnerable to <a href="http://en.wikipedia.org/wiki/Overfitting">overfitting</a>. To remedy this we can examine a plot of accuracy over increasing `k`. The optimal `k` is not often the highest `k` values, but rather the `k` at which improvements in accuracy begin to slow down dramatically.

Let's see what I mean here, and grab a dataset with many more values. Here we will use a few features of the wine ingredients to predict if it is high quality or not.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

df = pd.read_csv("https://s3.amazonaws.com/demo-datasets/wine.csv")

In [ ]:
# Take a look at the data
df.head()

In [ ]:
def best_k(df, features):
    # Randomly sample 70% training data, 30% testing data
    test_idx = np.random.uniform(0,1, len(df)) <= 0.3
    train = df[test_idx==True]
    test = df[test_idx==False]

    # List of features we want to test
    features = ['density', 'sulphates', 'residual_sugar']
    results = []
    for n in range(1,51,2):
        clf = KNeighborsClassifier(n_neighbors=n)
        clf.fit(train[features], train['high_quality'])
        preds = clf.predict(test[features])
        # If True = 1, if False = 0
        accuracy = np.where(preds==test['high_quality'], 1, 0).sum() / float(len(test))

        results.append([n, accuracy])

    results = pd.DataFrame(results, columns = ['n', 'accuracy'])
    
    plot = ggplot(results, aes("n", "accuracy"))+geom_line()
    return [results, test, train, plot]

# Features we want to use
features = ['density', 'sulphates', 'residual_sugar']

optimal_k_model = best_k(df, features)
optimal_k_model[3]

As we can see from this plot, the `k` with the best accuracy can be found at around **37**, however, accuracy doesn't really improve much after `k = ~9`. Most of the trend is caught at this `k` level, and beyond that we risk overfitting. Again, there are more sophisticated ways of doing this, but that will be discussed later.

In general, a larger `k` suppresses the effects of the noise, but makes the classification boundaries less distinct, which is really our actual goal.
<hr>

## Distance between points

Remember, KNN classification utilizes the idea that each observation is represented by it's features in N-dimensional space. Therefore we can compute the distance between the observations (in several ways). The second most important parameter in KNN models is how you choose to calculate the distance between the points.

The most common distance functions are `uniform`, which weights everything equally, and `distance`, which weights the points based on the inverse distance between them. There are many other ways to do this as well. 

Let's compare the results of all three of these types of weights:

In [ ]:
def KNN(train_data, test_data, k):
    results = []

    for w in ['uniform','distance', lambda x: np.log(x)]:
        # Run with 9 neighbors
        clf = KNeighborsClassifier(k, weights=w)
        w = str(w)
        clf.fit(train_data[features], train_data['high_quality'])
        prediction = clf.predict(test_data[features])
        accuracy = np.where(prediction==test_data['high_quality'], 1, 0).sum() / float(len(test_data))

        results.append([w, accuracy])

    results = pd.DataFrame(results, columns = ['weight_method', 'accuracy'])

    return results

train = optimal_k_model[2]
test = optimal_k_model[1]

KNN(train, test, 9)

Pretty slick. Though ideally, we would select the weighting scheme based on some intuition about our data. Should there be more feature influence at shorter distances? When it doubt, inverse distance is a good place to start, because it helps avoid problems of "majority voting" where one class dominates the data.

## Standardization

You may have noticed something about these above examples. We have been mixing up our units (this is bad) by summing up volumes, weights, densities and concentrations.

Let's take a look at our data again to understand whats happening here.

In [ ]:
df[features].describe()

If we are computing distance between these features in 3D space, we can see that small changes in density and sulphates, will equate to large changes in residual sugar content. This, in essence, makes residual sugar seem more important if we are treating the units equally, since the numbers are simply larger.

The solution is **standardization**, commonly referred to as **normalization** in statistics, whereby we adjust features on different scales to a single common scale. There are also various types of normalization, <a href="http://en.wikipedia.org/wiki/Normalization_(statistics)">see here</a>.

A common technique for normalizing **Z-score** (standard-score) values, which measures how far away from the mean a value is in terms of units of standard deviation. To do this we subtract the mean of each feature, then scale it by dividing it by their standard deviation. This can be implemented quite easily by hand, or also by using **`scale`** from `sklearns's` **`preprocessing`**  library.

In [ ]:
from sklearn import preprocessing

# Let's standardize our data from before we put in into our functions
# Remove color column because it is text and we arnt using it anyways.
#del df['color']
tmp = preprocessing.scale(df[features])
# Re add the column names back
df_norm = pd.DataFrame(tmp, columns = list(df[features].columns.values))
# Re-join the "High Quality" feature we are trying to predict
df_norm['high_quality'] = df['high_quality'] 

# Ignore some pesky warning about color not being an integer, thanks anyways pandas.
import warnings
warnings.simplefilter(action = "ignore", category = DeprecationWarning)

df_norm.describe()

Now lets repeat our KNN classification we did above with our newly standardized data and see what happens.

In [ ]:
# Find optimal k value
optimal_k_model = best_k(df_norm, features)
# Display the plot
optimal_k_model[3]

Looks like our `k` is still pretty close to before standardization, where a `k = ~9` appears to capture most of the trend.

In [ ]:
# Fit KNN function with different weights
# Extract the data from optimal_k_model
train = optimal_k_model[2]
test = optimal_k_model[1]
KNN(train, test, 9)

Great! We can see that our model improves after standardization, this is because the distances are more accurately abstracted in 3D space.

Lets go full circle now and predict if a **new** wine will be high quality or not using our model.

In [ ]:
# Sulphates, Residual sugar, and density of two new unseen wine to pass to our model
new_wine_features = [[0.66, 0.6, 0.92], [2.66, 2.1, 0.8]]

# Going through the motions again for repitition, 
# normally we would just take our model computed from above

train = optimal_k_model[2]
test = optimal_k_model[1]

clf = KNeighborsClassifier(9, "distance")
clf.fit(train[features], train['high_quality'])

print clf.get_params()
print clf.predict(new_wine_features)

Where 0 is not high quality and 1 is high quality.

In [ ]:
from IPython.core.display import HTML


def css_styling():
    styles = open("/users/ryankelly/desktop/custom_notebook.css", "r").read()
    return HTML(styles)
css_styling()